In [43]:
import pandas as pd
import numpy as np

In [44]:
# 打者の聞き手属性:右,左
BATTER_HANDEDNESS = 'batter_handedness'
BATTER_HANDEDNESS_ATTERRS = '右'

# 投手の聞き手属性:右,左
PITCHER_HANDEDNESS = 'pitcher_handedness'
PITCHER_HANDEDNESS_ATTERRS = '右'

# 球種:ストレート,スライダー,フォークカットボール,シュート,チェンジアップ,カーブ,シンカー,特殊球
PITCH_TYPE_NAME = 'pitch_type_name'
PITCH_TYPE_NAME_ATTRS = 'ストレート'

# コース:
#|21|19|17|15|10|
#|22|7 |4 |1 |11|
#|23|8 |5 |2 |12|
#|24|9 |6 |3 |13|
#|25|20|18|16|14|
PITCH_ZONE = 'pitch_zone'
SEARCH_COLUMN = '5'

# 球速:58.0〜163.0
PITCH_SPEED= 'pitch_speed'

#ストライク
IS_STRIKE = 'is_strike'

# 空振りフラグ:1
IS_CALLED_STRIKE = 'is_called_strike'

# 見逃しフラグ:1
IS_SWINGING_STRIKE = 'is_swinging_strike'

# スイングフラグ:1
IS_FOUL = 'is_foul'

# ストライクフラグ:1
IS_STRIKE = 'is_strike'

# 牽制フラグ:1
PICKOFF_ATTEMPT_TO = 'pickoff_attempt_to'

In [54]:
# ===== データの読み込み =====
# train.csv: モデルを学習するためのデータ
df = pd.read_csv('/Users/takurateruyoshi/Downloads/work_dataset/play_info.csv', encoding='cp932')

# フィルタリングを実行
# 1. 比較対象の列を文字列に変換 (.astype(str))
# 2. 比較 (== SEARCH_ATTRTS)
# 3. フィルタリング (train_df[...])
print(f"データ件数: {len(df)}")
df = df[df[PICKOFF_ATTEMPT_TO].astype(str).isin(['0'])]  # 牽制球のみ
print(f"データ件数: {len(df)}")

/var/folders/6t/2qvj7grs6tx1rjydtydst6br0000gn/T/ipykernel_46907/2145991319.py:3: DtypeWarning: Columns (125,141) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/takurateruyoshi/Downloads/work_dataset/play_info.csv', encoding='cp932')


データ件数: 254361
データ件数: 247574


In [46]:
# 文字列 "1"/"0" や True/False を 1/0 に正規化
s = df[IS_STRIKE]

# 文字→数値に変換（失敗は NaN）
s_num = pd.to_numeric(s, errors="coerce")

# True/False を 1/0 に
s_num = s_num.fillna(s.where(s.isin([True, False])).astype("float"))

# 以降は同様
n1 = (s_num == 1).sum()
n0 = (s_num == 0).sum()
valid = n1 + n0
ratio_1_valid = n1 / valid if valid > 0 else float("nan")

print(f"1の数: {n1}, 0の数: {n0}, 1の割合(有効): {ratio_1_valid:.3f}")

1の数: 161593, 0の数: 85981, 1の割合(有効): 0.653


In [64]:
#ストライク率の計算
for 投手 in df[BATTER_HANDEDNESS].unique():
    df_投手 = df[df[BATTER_HANDEDNESS].astype(str).isin([投手])]
    #print(f"投手: {投手}")
    for 打者 in df_投手[PITCHER_HANDEDNESS].unique():
        df_投手_打者 = df_投手[df_投手[PITCHER_HANDEDNESS].astype(str).isin([打者])]
        #print(f" 打者: {打者}")
        for 球種 in df_投手_打者[df_投手_打者[PITCH_TYPE_NAME].notna()][PITCH_TYPE_NAME].unique():
            df_投手_打者_球種 = df_投手_打者[df_投手_打者[PITCH_TYPE_NAME].astype(str).isin([球種])]
            #print(f"  球種: {球種}")
            for コース in df_投手_打者_球種[df_投手_打者_球種[PITCH_ZONE].notna()][PITCH_ZONE].unique():
                df_投手_打者_球種_コース = df_投手_打者_球種[df_投手_打者_球種[PITCH_ZONE].astype(str).isin([str(コース)])]
                #print(f"   コース: {コース}")
                投球合計数 = len(df_投手_打者_球種_コース)
                print(f"投手: {投手}, 打者: {打者}, 球種: {球種}, コース: {コース}")
                for 球速 in range(60, 161, 10):
                    df_投手_打者_球種_コース_球速 = df_投手_打者_球種_コース[
                        (df_投手_打者_球種_コース[PITCH_SPEED] >= 球速) &
                        (df_投手_打者_球種_コース[PITCH_SPEED] < 球速 + 10)
                    ]
                    投球数 = len(df_投手_打者_球種_コース_球速)
                    if 投球数 == 0:
                        continue
                    ストライク数 = df_投手_打者_球種_コース_球速[IS_STRIKE].sum()
                    ストライク率 = ストライク数 / 投球数
                    print(f"    球速: {球速}〜{球速 + 9} mph, 投球数: {投球数}, ストライク数: {ストライク数}, ストライク率: {ストライク率:.3f}")

投手: 左, 打者: 右, 球種: ストレート, コース: 2.0
    球速: 120〜129 mph, 投球数: 7, ストライク数: 7, ストライク率: 1.000
    球速: 130〜139 mph, 投球数: 50, ストライク数: 50, ストライク率: 1.000
    球速: 140〜149 mph, 投球数: 1543, ストライク数: 1543, ストライク率: 1.000
    球速: 150〜159 mph, 投球数: 972, ストライク数: 972, ストライク率: 1.000
    球速: 160〜169 mph, 投球数: 3, ストライク数: 3, ストライク率: 1.000
投手: 左, 打者: 右, 球種: ストレート, コース: 8.0
    球速: 110〜119 mph, 投球数: 1, ストライク数: 1, ストライク率: 1.000
    球速: 120〜129 mph, 投球数: 7, ストライク数: 7, ストライク率: 1.000
    球速: 130〜139 mph, 投球数: 32, ストライク数: 32, ストライク率: 1.000
    球速: 140〜149 mph, 投球数: 1010, ストライク数: 1010, ストライク率: 1.000
    球速: 150〜159 mph, 投球数: 692, ストライク数: 692, ストライク率: 1.000
    球速: 160〜169 mph, 投球数: 3, ストライク数: 3, ストライク率: 1.000
投手: 左, 打者: 右, 球種: ストレート, コース: 3.0
    球速: 120〜129 mph, 投球数: 10, ストライク数: 10, ストライク率: 1.000
    球速: 130〜139 mph, 投球数: 40, ストライク数: 40, ストライク率: 1.000
    球速: 140〜149 mph, 投球数: 1195, ストライク数: 1195, ストライク率: 1.000
    球速: 150〜159 mph, 投球数: 716, ストライク数: 716, ストライク率: 1.000
    球速: 160〜169 mph, 投球数: 1, ストライク数: 1, ストライク率: 1.